In [145]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import roc_curve, roc_auc_score, auc, matthews_corrcoef

import sys
import time
import pandas as pd

import math

from numpy import array, concatenate, dot, diag, mean, std, median


In [146]:
train_f = "DE_train.csv"

In [147]:


df_data = pd.read_csv(train_f)




In [148]:
Rec_m = list(df_data['RecA'])
Rec_m += list(df_data['RecB'])
Rec_v = []
for e in Rec_m:
    if e != "None":
        Rec_v.append(e)
        
Rec_v  = [float(x) for x in Rec_v]
Rec_m = median(Rec_v)



In [149]:
df_data.loc[df_data.RecA == 'None', 'RecA'] = Rec_m
df_data.loc[df_data.RecB == 'None', 'RecB'] = Rec_m


In [150]:
df_data

,ID,CADD1,CADD2,RecA,EssA,CADD3,CADD4,RecB,EssB,Path,DE,GP
0,dd239,4.304545,-1.0,0.24901,0,3.186035,-1.000000,0.16819,0,1,TD,MAN1B1/SEC23A
1,dd229,3.774086,-1.0,0.08467,1,4.959009,-1.000000,0.09998,1,1,TD,CDK5RAP2/CEP152
2,dd268,2.540195,-1.0,0.10394,1,-0.084020,-1.000000,0.24901,1,0,TD,LRP5/LY86-AS1
3,dd049,5.291170,-1.0,0.2097,1,1.851966,-1.000000,0.24901,1,1,CO,FZD4/F5
4,dd222,4.148268,-1.0,0.21357,0,3.754474,-1.000000,0.2303,0,1,TD,PRPH2/ROM1
...,...,...,...,...,...,...,...,...,...,...,...,...
327,du90,3.380462,-1.0,0.25081,0,2.170131,-1.000000,0.87681,0,1,DD,SMC1A/CFTR
328,du91,3.532409,-1.0,0.58893,1,4.509158,-1.000000,0.58549,1,0,DD,AVPR2/SLC45A2
329,du93,4.317497,-1.0,0.12564,1,5.559382,2.461791,0.13039,1,0,DD,HDAC8/IRX5
330,du94,4.229748,-1.0,0.22678,0,7.031820,4.195388,0.6537,0,0,DD,NLGN4X/BCS1L


In [151]:
X = (array(df_data)[:,1:-2].astype(float))
y = array(
        df_data['DE'].replace(
            'TD', 2
        ).replace(
            'CO', 1
        ).replace(
            'UK', -1
        ).replace(
            'OV', 0
        ).replace(
            'DI', 0
        ).replace(
            'DD', 0
        )
    )

gene_pairs = array(df_data['GP'])
#    X_pred = array(df_prediction)[:,1:11]


X, y, gene_pairs = X[y != -1], y[y != -1], gene_pairs[y != -1]

In [63]:
gene_pairs

array(['BBS2/MKKS', 'BBS2/MKKS', 'BBS2/MKKS', 'BBS2/MKKS', 'BBS4/BBS2',
       'BBS1/BBS7', 'BBS1/BBS2', 'BBS1/MKKS', 'BBS1/MKKS', 'BBS1/BBS2',
       'BBS1/BBS4', 'BBS2/BBS4', 'BBS2/BBS4', 'BBS1/CCDC28B',
       'BBS1/CCDC28B', 'BBS7/BBS4', 'BBS7/BBS2', 'BBS10/BBS5',
       'BBS10/MKKS', 'BBS10/BBS12', 'BBS10/BBS12', 'BBS10/MKKS',
       'BBS12/BBS4', 'BBS1/BBS10', 'BBS1/BBS10', 'BBS2/MKKS', 'BBS2/MKKS',
       'BBS10/BBS2', 'BBS2/BBS12', 'BBS2/BBS12', 'BBS10/MKKS',
       'BBS10/BBS2', 'BBS10/MKKS', 'BBS1/BBS2', 'BBS1/TTC8', 'BBS1/BBS9',
       'BBS10/BBS12', 'BBS10/MKKS', 'BBS10/BBS2', 'ARL6/TTC8',
       'BBS7/TTC8', 'BBS1/BBS12', 'BBS1/CCDC28B', 'NEK1/DYNC2H1',
       'MYH7/RBM20', 'DSP/TTN', 'DSP/TTN', 'MYH7/NEXN', 'NEXN/TTN',
       'GDAP1/MFN2', 'GDAP1/MFN2', 'EMD/LMNA', 'GNRHR/FGFR1',
       'OTUD4/RNF216', 'KAL1/FGFR1', 'KAL1/PROKR2', 'GNRHR/PROK2',
       'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9',
       'HAMP/HFE', 'HAMP/HFE', 'EDA/WNT10A', 'EDA/WNT

In [152]:
def getScores(pred, real, thresholds=[1, 1, 1]):
    """
    Returns evaluation metrics to evaluate one cross-validation:
    For each class, Sensitivity and Specificity. Order:
    sen_dd, sen_co, sen_td, spe_dd, spe_co, spe_td
    pred: Predicted probabilities. For each sample, vector is such as
    [pred_dd, pred_co, pred_td]
    real: real label. A label is a 3-long boolean vector.
    DD: [1, 0, 0] - CO: [0, 1, 0] - TD: [0, 0, 1]
    thresholds: weightings to compensate lack of data in certain class.
    """
    if len(pred) != len(real):
        print("pred ",len(pred), " real ", len(real))
        
        raise Exception("ERROR: input vectors have differente len!")
    results = {
        'sen': [ { 'count': 0, 'recognized': 0 } for _ in range(3) ],
        'spe': [ { 'count': 0, 'true': 0 } for _ in range(3) ],
    }
    for i, r_tab in enumerate(real):
        r = max(range(3), key=lambda k: r_tab[k])
        p = max(range(3), key=lambda k: pred[i][k]*thresholds[k])
        results['sen'][r]['count'] += 1
        results['spe'][p]['count'] += 1
        if p == r:
            results['sen'][p]['recognized'] += 1
            results['spe'][p]['true'] += 1
    return map(
        lambda x: round(x*100)/100,
        [r['recognized'] / r['count'] for r in results['sen']] + [r['true'] / r['count'] for r in results['spe']]
    )


def LOGO_crossValidation(X, y, groups, n_trees=100, n_epochs=50, thresholds=[1,1,1]):
    """
    Stratified cross-validation.
    X: Design matrix
    y: label vector
    groups: Gene pair vector to define training groups
    n_trees: Amount of trees in random forest
    n_epochs: number of cross validations to perform
    thresholds: see getScores
    """
    logo = LeaveOneGroupOut()
    clf = RandomForestClassifier(
        n_estimators=n_trees,
        max_depth=10,
        criterion='gini',
        min_samples_split=2,
        min_samples_leaf=2,
        bootstrap=True,
        n_jobs=1
    )
    # Vector to compute final scores
    sum_dd_se, sum_co_se, sum_td_se = [], [], []
    sum_dd_sp, sum_co_sp, sum_td_sp = [], [], []
    for i in range(n_epochs):
        start_time = time.time()
        values_t, values_p = [], []
        print("#"*10, "Trial %i" % i, "#"*10)
        # We leave one group out
        for train_index, test_index in logo.split(X, y, groups):
            X_fit, y_fit, X_train, y_train = (
                X[train_index], y[train_index],
                X[test_index],  y[test_index]
            )
            clf = clf.fit(X_fit, y_fit)
            y_predicted = clf.predict_proba(X_train)

            y_formatted = [ [0, 0, 0] for _ in range(len(y_predicted)) ]
            for i, e in enumerate(y_train):
                y_formatted[i][e] = 1
                

            values_t+= y_formatted
            for p in y_predicted:
                values_p += [[yi for yi in p]]
         
        
        sen_dd, sen_co, sen_td, spe_dd, spe_co, spe_td = getScores(values_p, values_t, thresholds)
        sum_dd_se.append(sen_dd)
        sum_co_se.append(sen_co)
        sum_td_se.append(sen_td)
        sum_dd_sp.append(spe_dd)
        sum_co_sp.append(spe_co)
        sum_td_sp.append(spe_td)
        print('Duration:', round( (time.time() - start_time) * 100) / 100, 's')
        print('sen | dd - co - td / spe | dd - co - td')
        print('sen | ' + '-'.join(map(str, (sen_dd, sen_co, sen_td))) + ' / spe | ' + '-'.join(map(str, (spe_dd, spe_co, spe_td))))
    print('Sen DD: %f, std: %f' % (mean(sum_dd_se), std(sum_dd_se)) )
    print('Sen CO: %f, std: %f' % (mean(sum_co_se), std(sum_co_se)) )
    print('Sen TD: %f, std: %f' % (mean(sum_td_se), std(sum_td_se)) )
    print('Spe DD: %f, std: %f' % (mean(sum_dd_sp), std(sum_dd_sp)) )
    print('Spe CO: %f, std: %f' % (mean(sum_co_sp), std(sum_co_sp)) )
    print('Spe TD: %f, std: %f' % (mean(sum_td_sp), std(sum_td_sp)) )
    geo_mn = (
        mean(sum_dd_se)*mean(sum_co_se)*mean(sum_td_se)*
        mean(sum_dd_sp)*mean(sum_co_sp)*mean(sum_td_sp)
    )**(1/6)
    geo_std = (
        std(sum_dd_se)*std(sum_co_se)*std(sum_td_se)*
        std(sum_dd_sp)*std(sum_co_sp)*std(sum_td_sp)
    )**(1/6)
    print('Geometric mean: %f, std: %f' % (geo_mn, geo_std))

In [153]:
LOGO_crossValidation(X,y, gene_pairs)

########## Trial 0 ##########
Duration: 24.03 s
sen | dd - co - td / spe | dd - co - td
sen | 0.66-0.45-0.58 / spe | 0.56-0.58-0.55
########## Trial 1 ##########
Duration: 21.89 s
sen | dd - co - td / spe | dd - co - td
sen | 0.66-0.51-0.63 / spe | 0.59-0.63-0.6
########## Trial 2 ##########
Duration: 21.91 s
sen | dd - co - td / spe | dd - co - td
sen | 0.62-0.55-0.56 / spe | 0.56-0.58-0.57
########## Trial 3 ##########
Duration: 21.87 s
sen | dd - co - td / spe | dd - co - td
sen | 0.63-0.47-0.59 / spe | 0.55-0.57-0.58
########## Trial 4 ##########
Duration: 22.92 s
sen | dd - co - td / spe | dd - co - td
sen | 0.7-0.51-0.61 / spe | 0.6-0.63-0.6
########## Trial 5 ##########
Duration: 24.07 s
sen | dd - co - td / spe | dd - co - td
sen | 0.66-0.51-0.62 / spe | 0.57-0.6-0.62
########## Trial 6 ##########
Duration: 21.69 s
sen | dd - co - td / spe | dd - co - td
sen | 0.63-0.51-0.57 / spe | 0.53-0.59-0.59
########## Trial 7 ##########
Duration: 20.68 s
sen | dd - co - td / spe | dd - c

In [139]:
train_f = "DE_train_prev"

In [141]:
df_data = pd.read_csv(train_f)

In [142]:
X = (array(df_data)[:,1:-2].astype(float))
y = array(
        df_data['DE'].replace(
            'TD', 2
        ).replace(
            'CO', 1
        ).replace(
            'UK', -1
        ).replace(
            'OV', 0
        ).replace(
            'DI', 0
        ).replace(
            'DD', 0
        )
    )

gene_pairs = array(df_data['GP'])
#    X_pred = array(df_prediction)[:,1:11]


X, y, gene_pairs = X[y != -1], y[y != -1], gene_pairs[y != -1]

In [144]:
LOGO_crossValidation(X,y, gene_pairs)

########## Trial 0 ##########
Duration: 25.22 s
sen | dd - co - td / spe | dd - co - td
sen | 0.81-0.57-0.71 / spe | 0.81-0.65-0.65
########## Trial 1 ##########
Duration: 21.06 s
sen | dd - co - td / spe | dd - co - td
sen | 0.79-0.55-0.7 / spe | 0.77-0.64-0.64
########## Trial 2 ##########
Duration: 25.65 s
sen | dd - co - td / spe | dd - co - td
sen | 0.8-0.6-0.72 / spe | 0.77-0.67-0.68
########## Trial 3 ##########
Duration: 21.88 s
sen | dd - co - td / spe | dd - co - td
sen | 0.8-0.59-0.72 / spe | 0.8-0.64-0.68
########## Trial 4 ##########
Duration: 21.2 s
sen | dd - co - td / spe | dd - co - td
sen | 0.8-0.49-0.73 / spe | 0.79-0.62-0.63
########## Trial 5 ##########
Duration: 20.58 s
sen | dd - co - td / spe | dd - co - td
sen | 0.79-0.55-0.67 / spe | 0.78-0.59-0.63
########## Trial 6 ##########
Duration: 20.84 s
sen | dd - co - td / spe | dd - co - td
sen | 0.8-0.56-0.72 / spe | 0.78-0.67-0.65
########## Trial 7 ##########
Duration: 21.41 s
sen | dd - co - td / spe | dd - co -